In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Activation
from tensorflow.keras.layers import concatenate, Dropout, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
x = train.drop(['id', 'digit', 'letter'], axis=1).values
x = x.reshape(-1, 28, 28, 1)
x = x/255

y_data = train['digit']
y = np.zeros((len(y_data), len(y_data.unique())))
for i, digit in enumerate(y_data):
    y[i, digit] = 1

In [4]:
x_letter = train['letter'].values
x_letter = x_letter[:, np.newaxis]
en = OneHotEncoder()
x_letter = en.fit_transform(x_letter).toarray()

In [5]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, shuffle=True)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

x_letter_train = x_letter[:x_train.shape[0],:]
x_letter_val = x_letter[x_train.shape[0]:,:]
print(x_letter_train.shape)
print(x_letter_val.shape)

(1843, 28, 28, 1)
(205, 28, 28, 1)
(1843, 10)
(205, 10)
(1843, 26)
(205, 26)


In [6]:
def Conv_block(x, growth_rate, activation='relu'):
    x_l = BatchNormalization()(x)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate*4, (1,1), padding='same', kernel_initializer='he_normal')(x_l)
    
    x_l = BatchNormalization()(x_l)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate, (3,3), padding='same', kernel_initializer='he_normal')(x_l)
    
    x = concatenate([x, x_l])
    return x

def Dense_block(x, layers, growth_rate=32):
    for i in range(layers):
        x = Conv_block(x, growth_rate)
    return x

def Transition_layer(x, compression_factor=0.5, activation='relu'):
    reduced_filters = int(tf.keras.backend.int_shape(x)[-1] * compression_factor)
    
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(reduced_filters, (1,1), padding='same', kernel_initializer='he_normal')(x)
    
    x = AveragePooling2D((2,2), padding='same', strides=2)(x)
    return x

def DenseNet(model_input, classes, densenet_type='DenseNet-121'):
    x = Conv2D(base_growth_rate*2, (5,5), padding='same', strides=1,
               kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2,2), padding='same', strides=2)(x)
    
    x = Dense_block(x, layers_in_block[densenet_type][0], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][1], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][2], base_growth_rate)
    #x = Transition_layer(x, compression_factor=0.5)
    #x = Dense_block(x, layers_in_block[densenet_type][3], base_growth_rate)
    
    x = GlobalAveragePooling2D()(x)
    
    model_output = Dense(classes, activation='softmax', kernel_initializer='he_normal')(x)
    
    model = Model(model_input, model_output, name=densenet_type)
    
    return model

In [7]:
def DenseNet_letter(model_input, letter, classes, densenet_type='DenseNet-121'):
    x = Conv2D(base_growth_rate*2, (5,5), padding='same', strides=1,
               kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2,2), padding='same', strides=2)(x)
    
    x = Dense_block(x, layers_in_block[densenet_type][0], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][1], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][2], base_growth_rate)
    #x = Transition_layer(x, compression_factor=0.5)
    #x = Dense_block(x, layers_in_block[densenet_type][3], base_growth_rate)
    
    x = GlobalAveragePooling2D()(x)
    
    merge = concatenate([x, letter])
    x1 = Dense(500, activation='relu')(merge)
    x1 = Dropout(0.3)(x1)
    
    model_output = Dense(classes, activation='softmax', kernel_initializer='he_normal')(x1)
    
    model = Model(inputs = [model_input, letter], outputs = model_output, name=densenet_type)
    
    return model

In [8]:
layers_in_block = {'DenseNet-121':[6, 12, 24, 16],
                   'DenseNet-169':[6, 12, 32, 32],
                   'DenseNet-201':[6, 12, 48, 32],
                   'DenseNet-265':[6, 12, 64, 48],
                   'myDenseNet':[8, 16, 24, 32]}

base_growth_rate = 32

model_input = Input(shape=(28,28,1))
letter = Input(shape=(26,))
classes = 10

model = DenseNet_letter(model_input, letter, classes, 'myDenseNet')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   1664        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 28, 28, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 28, 28, 64)   0           batch_normalization[0][0]        
__________________________________________________________________________________________________
max_poolin

In [9]:
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy']) 

cp = ModelCheckpoint('./models/{epoch:02d}-{val_acc:.4f}.h5', monitor='val_loss',
                     save_best_only=True, mode='min')

history = model.fit([x_train, x_letter_train], y_train, validation_data=([x_val, x_letter_val], y_val),
                    batch_size=64, epochs=100, callbacks = [cp])

Train on 1843 samples, validate on 205 samples
Epoch 1/100
1843/1843 [==============================] - 42s 23ms/step - loss: 2.3494 - acc: 0.1959 - val_loss: 2.3056 - val_acc: 0.0976
Epoch 2/100
1843/1843 [==============================] - 10s 5ms/step - loss: 1.7401 - acc: 0.3760 - val_loss: 2.6861 - val_acc: 0.1073
Epoch 3/100
1843/1843 [==============================] - 10s 5ms/step - loss: 1.2994 - acc: 0.5431 - val_loss: 2.8629 - val_acc: 0.0927
Epoch 4/100
1843/1843 [==============================] - 10s 5ms/step - loss: 0.9460 - acc: 0.6690 - val_loss: 7.6233 - val_acc: 0.0976
Epoch 5/100
1843/1843 [==============================] - 10s 5ms/step - loss: 0.7281 - acc: 0.7651 - val_loss: 12.2823 - val_acc: 0.0927
Epoch 6/100
1843/1843 [==============================] - 10s 5ms/step - loss: 0.5791 - acc: 0.8226 - val_loss: 12.1275 - val_acc: 0.1073
Epoch 7/100
1843/1843 [==============================] - 10s 5ms/step - loss: 0.5820 - acc: 0.8068 - val_loss: 7.3365 - val_acc: 0.107